In [4]:
#import libraries
import random
import sqlite3
from sqlite3 import Error

#define shopping listS
shoppinglist = ['bread','milk','eggs','chicken','beef','pork','tofu','noodles','rice','water','juice','milk','coffee','tea','lettuce','watermelon','bananas','tomatoes','pickles','lemons','peas','cabbage','chili','cashews','edamame','carrots','radish','beer','papertowels','napkins']
len(shoppinglist)

30

In [ ]:
#generate 20 transactions
def generate_transactions(inputlist):
    index = 0
    transactions = []
    while index < 20:
        
        #generate random number of items in each shopping transaction
        transnum = random.randrange(1,20,1)

        #generate transactions using the transnum
        transaction = random.sample(inputlist,transnum)

        #sort alphabetically and append to transactions list
        transaction.sort()
        transactions.append(transaction)

        index +=1
    
    return transactions

#show transactions
transactions = generate_transactions(shoppinglist)
for listitem in range(len(transactions)):
    print(transactions[listitem])
    if 'cabbage' in transactions[listitem]:
        if 'lettuce' in transactions[listitem]:
            print(True)

In [5]:
#create database(s) and relevant tables

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def create_table(conn,name):
    """ create table from the create_table_statement"""
    try:
        c = conn.cursor()
        c.execute("""CREATE TABLE IF NOT EXISTS %s (item_id text PRIMARY KEY)""" %(name))
        print('Table created')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)
    

def populate_table(conn,tablename,itemname):
    """populate table from populate_statement"""
    try:
        c = conn.cursor()
        c.execute("""INSERT INTO %s (item_id) VALUES (?)""" %(tablename),[itemname])
        print('Table populated')
        conn.commit()
        conn.close()
    except Error as e:
        print(e)


def query(conn,statement):
    try:
        c = conn.cursor()
        c.execute(statement)
        conn.commit()
        row = c.fetchall()
        print(row)
        conn.close()
    except Error as e:
        print(e)
        
def main():
    database = r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db"
    
    index = 0
    #create the 20 transaction tables
    for index in range(len(transactions)):
        #create database connection
        conn = create_connection(database)
        if conn is not None:
            table_name = 'transaction_'+str(index)
            create_table(conn,table_name)
            #populate each table with the items in each respective transaction
            for item in transactions[index]:
                item_name = item
                conn = create_connection(database)
                populate_table(conn,table_name,item_name)
        else:
            print("Cannot connect to database")
            
#test
if __name__ == '__main__':
    #create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
    #main()
    statement = "SELECT * FROM transaction_4 ORDER BY item_id ASC"
    #statement = "UPDATE transaction_4 SET item_id = 'pork' WHERE item_id='edamame'"
    query(create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db"),statement)


2.6.0
[('cabbage',), ('milk',), ('napkins',), ('pork',)]


In [7]:
## Initialize the algorithm

In [264]:
#get transactions from database in list form
def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        c.execute("SELECT * FROM %s"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)
   

#create hash map with collision prevention and visited flags
def createHashMap(transaction):
    hash_map = [[] for i in range(100)]
    for item in transaction:
        
        #hash item value
        h = 0
        for char in item:
            h += ord(char)
            h = h % 50
        key = h
        if hash_map[key] is None:
            hash_map[key].append(item)
        else:
            hash_map[key].append(item)
    return hash_map


#get hashes of all items in list to use for finding the support
def hashShoppingList(shoplist):
    hashlist = []
    for item in range(len(shoplist)):
        hashlist.append(getHash(shoplist[item]))
    return hashlist

        
                
#test
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
result_list = getItems(conn,0)
hash_map = createHashMap(result_list)
items_hash = hashShoppingList(shoppinglist)

print(hash_map)
print(items_hash)


2.6.0


NameError: name 'getHash' is not defined

In [80]:
def generateGroups(input_set,size):
    input_set = list(input_set)
    if size == 0:
        return [[]]
    new_set = []
    for index in range(0,len(input_set)):
        item = input_set[index]
        remaining = input_set[index + 1:]
        for j in generateGroups(remaining,size-1):
            new_set.append([item]+list(j))
    return new_set

def unionSets(input_1,input_2):
    return set(input_1).union(set(input_2))

def getItems(conn,transnum):
    try:
        items_list = []
        c = conn.cursor()
        c.execute("SELECT item_id FROM %s ORDER BY item_id ASC"%('transaction_'+str(transnum)))
        transaction = c.fetchall()
        for item in transaction:
            items_list.append(item[0])
        return items_list
        conn.close()
    except Error as e:
        print(e)

def getTransactions(conn,translength):
        transactions = []
        for i in range(translength):
            transaction = getItems(conn,i)
            transactions.append(transaction)
        return transactions

def scanTransactions(itemset,transactions):
    count = 0
    for i in range(len(transactions)):
        if set(itemset).issubset(transactions[i]):
            count +=1
    return count   

def getUnions(set1,n):
    sets = []
    for item in set1:
        if type(item) == type(''):
            item = [item]
            for i in set1:
                if type(i) == type(''):
                    i = [i]
                    if item != i:
                        u = set(item).union(set(i))
                        if u not in sets and len(u) == n:
                            sets.append(u)
        else:
            itemset = list(item)
            #print(set(itemset))
            for j in set1:
                if type(j) != type((1,2)):
                    u = set(itemset).union({j})
                    if u not in sets and len(u) == n:
                        sets.append(u)
                else:
                    itemset2 = set(list(j))
                    u = itemset2.union(itemset)
                    if u not in sets and len(u) == n:
                        sets.append(u)
                      
    return sets

def apriori(set_list,min_support):
    #initialize
    discarded = {}
    C = {}
    L = {}
    stop = False
    n = 1
    while stop == False:
        if n <= 1:
            for transaction in range(len(set_list)):
                for item in set_list[transaction]:
                    #prevent duplicate scans of same subset
                    if item not in C:
                        #prevent scans of already discarded items and previously non-frequent sets
                        if item not in discarded:
                            sup_count = scanTransactions({item},set_list)
                            C[item] = sup_count

            #print first scan results
            print('\nIteration {}'.format(n))
            print('C1 candidate set: \n')
            
            for i in C:
                print(i,':',C[i])
            #create L with items that meet minimum support requirement
            for item in C:
                sup_calc = C[item] / len(set_list)
                if item not in discarded:
                    if sup_calc >= min_support:
                        L[item] = C[item]
                    else:
                        discarded[item] = sup_calc
                    
            print('\nL1 set: \n')
            for i in L:
                print(i,':',L[i])
            print('#'*75)
            
            L_n = L #helps to generate sets for second iteration 
            n += 1
            if L is None:
                stop = True
                break
        elif n > 1:
            C_n = {}
            #union of the sets
            sets = []
            L_n = list(L_n)
            sets = getUnions(L_n,n)
            
            #save last iteration's results to a variable
            L_n_minus_1 = L_n
            
            #reinitialize L_n for the following iteration
            L_n = {}
            
            for i in range(len(sets)):
                support_count = scanTransactions(sets[i],set_list)
                C_n[tuple(sets[i])] = support_count
            if C_n == {}:
                stop = True
                break
            
            #print candidate set
            print('\nIteration {}'.format(n))
            print('C{} candidate set: \n'.format(n))
            for i in C_n:
                print(i,':',C_n[i])
            
            #calculate supports of each item in candidate set and prune
            for item in C_n:
                #if itemset is frequent, then subsets must also be frequent (and not in discarded)
                if item not in discarded:
                    sup_calc = C_n[item] / len(set_list)
                    if sup_calc >= min_support:
                        L_n[item] = C_n[item]
                    else:
                        discarded[item] = sup_calc
             
            #print passing set
            print('\nL{} set: \n'.format(n))
            for i in L_n:
                print(i,':',L_n[i])
            print('#'*75)
            
            if L_n == {} or L_n is None:
                stop = True
                return L_n_minus_1
                break
            n += 1
    return L_n_minus_1
            
def makeAssociationRules(inputset,min_confidence,transactions):
    print("Association Rules that meet minimum confidence: \n")
    for i in inputset:
        i = list(i)
        for j in i:
            right = {j}
            left = set(i) - right
            both = i
            counts_both = scanTransactions(both,transactions)
            counts_left = scanTransactions(list(left),transactions)
            sup_both = counts_both / len(transactions)
            sup_left = counts_left / len(transactions)
            if sup_left != 0:
                if sup_both / sup_left >= min_confidence:
                    print(left,'->',right,'    ','confidence: ',(sup_both/sup_left),'\n')
            
conn = create_connection(r"C:\Users\Nick\CS634_Data_Mining\midtermproject.db")
transactionsList = getTransactions(conn,20)
results = apriori(transactionsList,0.4)
makeAssociationRules(results,.6,transactionsList)

2.6.0

Iteration 1
C1 candidate set: 

bananas : 6
cabbage : 13
carrots : 8
cashews : 5
juice : 6
milk : 12
pork : 11
radish : 6
rice : 5
watermelon : 6
beer : 8
chili : 5
lettuce : 11
noodles : 9
pickles : 7
tea : 10
tomatoes : 4
bread : 9
lemons : 6
napkins : 7
papertowels : 8
tofu : 6
chicken : 9
edamame : 5
water : 6
coffee : 5
peas : 9
beef : 6
eggs : 6

L1 set: 

cabbage : 13
carrots : 8
milk : 12
pork : 11
beer : 8
lettuce : 11
noodles : 9
tea : 10
bread : 9
papertowels : 8
chicken : 9
peas : 9
###########################################################################

Iteration 2
C2 candidate set: 

('carrots', 'cabbage') : 6
('cabbage', 'milk') : 8
('cabbage', 'pork') : 6
('beer', 'cabbage') : 6
('cabbage', 'lettuce') : 8
('cabbage', 'noodles') : 6
('cabbage', 'tea') : 7
('bread', 'cabbage') : 5
('cabbage', 'papertowels') : 6
('chicken', 'cabbage') : 6
('peas', 'cabbage') : 5
('carrots', 'milk') : 5
('carrots', 'pork') : 5
('beer', 'carrots') : 3
('carrots', 'lettuce') : 3
('

In [81]:
##brute force method

def bruteForce(set_list,min_support):
    n = 1
    C = {}
    L = {}
    stop = False
    while stop == False:
        if n <= 1:
            for transaction in range(len(set_list)):
                for item in set_list[transaction]:
                    #prevent duplicate scans of same subset
                    if type(item) != type(['type']):
                        sup_count = scanTransactions({item},set_list)
                        C[item] = sup_count
                    else:
                        sup_count = scanTransactions(item,set_list)
                        C[item] = sup_count

            #print first results
            print('#' * 50)
            print('\nIteration {} Set:\n'.format(n))

            for i in C:
                print(i,':',C[i])
            C_n = C
            n += 1
        else:
            
            #union of the sets
            sets = []
            C_n = list(C_n)
            sets = getUnions(C_n,n)
            print(sets)
            #save last iteration's results to a variable
            C_n_minus_1 = C_n
            
            #reinitialize L_n for the following iteration
            C_n = {}
            
            for i in range(len(sets)):
                support_count = scanTransactions(sets[i],set_list)
                C_n[tuple(sets[i])] = support_count
            
            #print set
            #print('#' * 50)
            #print('\nIteration {}'.format(n))
            #print('C{} candidate set: \n'.format(n))
            
            #list that will hold a value if the minimum support is reached
            #if no sets meet the minimum support, the loop will break
            passing_supports = {}
            for i in C_n:
                print(i,':',C_n[i])
                if C_n[i] / len(set_list) >= min_support:
                    passing_supports[i] = C_n[i]
            
            #print('#' * 50)
            #print('#' * 50)
            #print(passing_supports)
            #print('#' * 50)
            #print('#' * 50)
            if passing_supports == {} or passing_supports is None:
                stop = True
                return C_n_minus_1
                break
            
            if C_n == {} or C_n is None:
                stop = True
                return C_n_minus_1
                break
            n += 1
            
def makeAssociationRulesBrute(inputset,min_support,min_confidence,transactions):
    print("Association Rules that meet minimum confidence: \n")
    for i in inputset:
        i = list(i)
        counts_i = scanTransactions(i,transactions)
        sup_i = counts_i / len(transactions)
        if sup_i >= min_support:
            for j in i:
                counts_j = scanTransactions({j},transactions)
                sup_j = counts_j / len(transactions)
                right = {j}
                left = set(i) - right
                both = i
                counts_both = scanTransactions(both,transactions)
                counts_left = scanTransactions(list(left),transactions)
                counts_right = scanTransactions(list(right),transactions)
                sup_both = counts_both / len(transactions)
                sup_left = counts_left / len(transactions)
                sup_right = counts_right / len(transactions)
                if sup_left != 0:
                    if sup_left >= min_support and sup_right >= min_support:
                        if sup_both / sup_left >= min_confidence:
                            print(left,'->',right,'    ','confidence: ',(sup_both/sup_left),'\n')    


brute_result = bruteForce(transactionsList,.4)
makeAssociationRulesBrute(brute_result,.4,.6,transactionsList)

##################################################

Iteration 1 Set:

bananas : 6
cabbage : 13
carrots : 8
cashews : 5
juice : 6
milk : 12
pork : 11
radish : 6
rice : 5
watermelon : 6
beer : 8
chili : 5
lettuce : 11
noodles : 9
pickles : 7
tea : 10
tomatoes : 4
bread : 9
lemons : 6
napkins : 7
papertowels : 8
tofu : 6
chicken : 9
edamame : 5
water : 6
coffee : 5
peas : 9
beef : 6
eggs : 6
[{'cabbage', 'bananas'}, {'carrots', 'bananas'}, {'cashews', 'bananas'}, {'juice', 'bananas'}, {'bananas', 'milk'}, {'pork', 'bananas'}, {'radish', 'bananas'}, {'rice', 'bananas'}, {'bananas', 'watermelon'}, {'beer', 'bananas'}, {'chili', 'bananas'}, {'lettuce', 'bananas'}, {'bananas', 'noodles'}, {'bananas', 'pickles'}, {'bananas', 'tea'}, {'tomatoes', 'bananas'}, {'bread', 'bananas'}, {'lemons', 'bananas'}, {'napkins', 'bananas'}, {'papertowels', 'bananas'}, {'bananas', 'tofu'}, {'chicken', 'bananas'}, {'edamame', 'bananas'}, {'bananas', 'water'}, {'coffee', 'bananas'}, {'peas', 'bananas'}, {'beef',

[{'carrots', 'cabbage', 'bananas'}, {'cashews', 'cabbage', 'bananas'}, {'juice', 'cabbage', 'bananas'}, {'cabbage', 'bananas', 'milk'}, {'bananas', 'cabbage', 'pork'}, {'radish', 'cabbage', 'bananas'}, {'rice', 'cabbage', 'bananas'}, {'cabbage', 'bananas', 'watermelon'}, {'beer', 'cabbage', 'bananas'}, {'chili', 'cabbage', 'bananas'}, {'bananas', 'cabbage', 'lettuce'}, {'cabbage', 'bananas', 'noodles'}, {'cabbage', 'bananas', 'pickles'}, {'cabbage', 'bananas', 'tea'}, {'tomatoes', 'cabbage', 'bananas'}, {'cabbage', 'bread', 'bananas'}, {'lemons', 'cabbage', 'bananas'}, {'napkins', 'cabbage', 'bananas'}, {'cabbage', 'papertowels', 'bananas'}, {'cabbage', 'bananas', 'tofu'}, {'chicken', 'cabbage', 'bananas'}, {'cabbage', 'edamame', 'bananas'}, {'cabbage', 'bananas', 'water'}, {'cabbage', 'coffee', 'bananas'}, {'peas', 'cabbage', 'bananas'}, {'bananas', 'cabbage', 'beef'}, {'cabbage', 'bananas', 'eggs'}, {'cashews', 'bananas', 'carrots'}, {'juice', 'carrots', 'bananas'}, {'carrots', 'bana

('cabbage', 'watermelon', 'noodles') : 2
('cabbage', 'watermelon', 'pickles') : 0
('cabbage', 'watermelon', 'tea') : 0
('tomatoes', 'cabbage', 'watermelon') : 0
('cabbage', 'bread', 'watermelon') : 1
('lemons', 'cabbage', 'watermelon') : 1
('napkins', 'cabbage', 'watermelon') : 2
('cabbage', 'watermelon', 'papertowels') : 1
('cabbage', 'tofu', 'watermelon') : 1
('chicken', 'cabbage', 'watermelon') : 2
('cabbage', 'watermelon', 'edamame') : 0
('watermelon', 'cabbage', 'water') : 0
('cabbage', 'coffee', 'watermelon') : 1
('peas', 'cabbage', 'watermelon') : 2
('cabbage', 'beef', 'watermelon') : 1
('watermelon', 'cabbage', 'eggs') : 2
('chili', 'cabbage', 'beer') : 4
('beer', 'cabbage', 'lettuce') : 4
('beer', 'cabbage', 'noodles') : 4
('beer', 'cabbage', 'pickles') : 2
('beer', 'cabbage', 'tea') : 5
('beer', 'tomatoes', 'cabbage') : 3
('beer', 'cabbage', 'bread') : 2
('beer', 'lemons', 'cabbage') : 1
('napkins', 'cabbage', 'beer') : 2
('beer', 'cabbage', 'papertowels') : 3
('beer', 'cabba

('juice', 'chili', 'beer') : 3
('juice', 'beer', 'lettuce') : 3
('juice', 'beer', 'noodles') : 3
('juice', 'beer', 'pickles') : 1
('juice', 'beer', 'tea') : 2
('juice', 'tomatoes', 'beer') : 2
('juice', 'bread', 'beer') : 0
('juice', 'lemons', 'beer') : 0
('juice', 'napkins', 'beer') : 1
('juice', 'papertowels', 'beer') : 1
('juice', 'beer', 'tofu') : 1
('juice', 'beer', 'chicken') : 2
('juice', 'edamame', 'beer') : 1
('juice', 'beer', 'water') : 1
('juice', 'coffee', 'beer') : 0
('juice', 'peas', 'beer') : 1
('juice', 'beer', 'beef') : 2
('juice', 'beer', 'eggs') : 1
('juice', 'chili', 'lettuce') : 3
('juice', 'chili', 'noodles') : 3
('juice', 'chili', 'pickles') : 1
('juice', 'chili', 'tea') : 2
('juice', 'chili', 'tomatoes') : 2
('juice', 'chili', 'bread') : 0
('juice', 'chili', 'lemons') : 0
('juice', 'napkins', 'chili') : 1
('juice', 'chili', 'papertowels') : 1
('juice', 'chili', 'tofu') : 1
('juice', 'chili', 'chicken') : 2
('juice', 'chili', 'edamame') : 1
('juice', 'chili', 'wa

('bread', 'pork', 'tea') : 3
('lemons', 'pork', 'tea') : 2
('napkins', 'pork', 'tea') : 1
('papertowels', 'pork', 'tea') : 3
('pork', 'tofu', 'tea') : 3
('chicken', 'pork', 'tea') : 3
('edamame', 'pork', 'tea') : 3
('pork', 'water', 'tea') : 3
('coffee', 'pork', 'tea') : 2
('peas', 'pork', 'tea') : 3
('pork', 'beef', 'tea') : 3
('pork', 'eggs', 'tea') : 2
('tomatoes', 'bread', 'pork') : 0
('lemons', 'tomatoes', 'pork') : 0
('napkins', 'tomatoes', 'pork') : 0
('tomatoes', 'papertowels', 'pork') : 1
('tomatoes', 'pork', 'tofu') : 1
('chicken', 'tomatoes', 'pork') : 1
('tomatoes', 'edamame', 'pork') : 1
('tomatoes', 'pork', 'water') : 1
('tomatoes', 'coffee', 'pork') : 0
('peas', 'tomatoes', 'pork') : 0
('pork', 'tomatoes', 'beef') : 1
('tomatoes', 'pork', 'eggs') : 1
('lemons', 'bread', 'pork') : 3
('napkins', 'bread', 'pork') : 2
('bread', 'papertowels', 'pork') : 2
('bread', 'pork', 'tofu') : 4
('chicken', 'bread', 'pork') : 3
('bread', 'edamame', 'pork') : 2
('bread', 'pork', 'water')

('beer', 'lettuce', 'beef') : 3
('beer', 'lettuce', 'eggs') : 2
('beer', 'noodles', 'pickles') : 2
('beer', 'noodles', 'tea') : 3
('beer', 'tomatoes', 'noodles') : 3
('beer', 'bread', 'noodles') : 1
('beer', 'lemons', 'noodles') : 1
('beer', 'napkins', 'noodles') : 3
('beer', 'papertowels', 'noodles') : 2
('beer', 'tofu', 'noodles') : 1
('beer', 'noodles', 'chicken') : 3
('beer', 'edamame', 'noodles') : 2
('beer', 'water', 'noodles') : 2
('beer', 'coffee', 'noodles') : 1
('beer', 'peas', 'noodles') : 2
('beer', 'beef', 'noodles') : 3
('beer', 'eggs', 'noodles') : 3
('beer', 'pickles', 'tea') : 2
('beer', 'tomatoes', 'pickles') : 1
('beer', 'bread', 'pickles') : 1
('beer', 'lemons', 'pickles') : 2
('beer', 'napkins', 'pickles') : 1
('beer', 'papertowels', 'pickles') : 1
('beer', 'tofu', 'pickles') : 1
('beer', 'pickles', 'chicken') : 1
('beer', 'edamame', 'pickles') : 2
('beer', 'water', 'pickles') : 1
('beer', 'coffee', 'pickles') : 2
('beer', 'peas', 'pickles') : 1
('beer', 'beef', 'p

('peas', 'chicken', 'tea') : 3
('chicken', 'beef', 'tea') : 2
('chicken', 'eggs', 'tea') : 2
('edamame', 'water', 'tea') : 3
('coffee', 'edamame', 'tea') : 1
('peas', 'edamame', 'tea') : 2
('edamame', 'beef', 'tea') : 2
('edamame', 'eggs', 'tea') : 2
('coffee', 'water', 'tea') : 2
('peas', 'water', 'tea') : 3
('beef', 'water', 'tea') : 2
('water', 'eggs', 'tea') : 2
('peas', 'coffee', 'tea') : 3
('coffee', 'beef', 'tea') : 1
('coffee', 'eggs', 'tea') : 0
('peas', 'beef', 'tea') : 2
('peas', 'eggs', 'tea') : 2
('beef', 'eggs', 'tea') : 2
('lemons', 'tomatoes', 'bread') : 0
('napkins', 'tomatoes', 'bread') : 1
('bread', 'tomatoes', 'papertowels') : 0
('tomatoes', 'bread', 'tofu') : 0
('chicken', 'tomatoes', 'bread') : 0
('bread', 'tomatoes', 'edamame') : 0
('tomatoes', 'bread', 'water') : 0
('bread', 'tomatoes', 'coffee') : 0
('peas', 'tomatoes', 'bread') : 1
('tomatoes', 'bread', 'beef') : 0
('tomatoes', 'bread', 'eggs') : 1
('napkins', 'lemons', 'tomatoes') : 0
('lemons', 'tomatoes', '